# Can we identify faulty or suboptimally performing equipment?

In [1]:
#setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

gd=pd.read_csv("Data/Train_Plant2.csv")

In [2]:
#Add columns
gd['DC_POWER_DIF_AVG_PER_ALL']=gd['DC_POWER']-gd['DC_POWER'].mean()
gd['AC_POWER_DIF_AVG_PER_ALL']=gd['AC_POWER']-gd['AC_POWER'].mean()
gd['DATE_TIME'] = pd.to_datetime(gd['DATE_TIME'], errors='coerce')
gd['DATE']=pd.to_datetime(gd['DATE_TIME'].dt.date)
gd['TIME']=gd['DATE_TIME'].dt.time

In [3]:
#New columns per inverter 
uurimiseks= gd.copy()
groups = dict(list(uurimiseks.groupby("SOURCE_KEY")))
shift_nr=3
shift_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']
gr_nr=0

#inverter based shifting
for group in groups:
    gr_nr+=1
    dates=groups[group]['DATE'].unique()
    for i in range(shift_nr):
        for j in range(len(shift_columns)):
            groups[group][shift_columns[j]+'_PER_INVERTER_MOMENT_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=-(i+1), fill_value=0)
            groups[group][shift_columns[j]+'_PER_INVERTER_DAY_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=-(i+1)*96, fill_value=0)
    
    #groups[group]['INVERTER']=group - ebaoluline
    groups[group]['DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    ############ per one day comparison, but note sure if correct
    gr_no2=0
    for date in dates:
        gr_no2 +=1
        d=groups[group][groups[group]['DATE']==date]
        d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
        d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])
        if gr_no2==1:
            date_data=pd.DataFrame(columns = d.keys())
            
        date_data =pd.concat([date_data, d], ignore_index=True)
    #########
    
    if gr_nr==1:
        gen_data=pd.DataFrame(columns = date_data.keys())
    
    #gen_data.append(groups[group], ignore_index=True) - did not seem to work and was replaced with concat
    gen_data=pd.concat([gen_data,date_data], ignore_index=True)
    
#gen_data=gen_data.fillna(value=0) - ebaoluline
gen_data

<ipython-input-3-d3be87962ed3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
<ipython-input-3-d3be87962ed3>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])


,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,AMBIENT_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,AMBIENT_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY
0,2020-05-15 00:00:00,4UPUqMRk7TRMgml,27.004764,25.060789,0.000000,4136001.0,0.000000,0.000000,9425.000000,2.429011e+06,...,26.500589,23.441600,24.420678,23.655607,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485
1,2020-05-15 00:15:00,4UPUqMRk7TRMgml,26.880811,24.421869,0.000000,4136001.0,0.000000,0.000000,9425.000000,2.429011e+06,...,26.596148,23.881824,25.088210,23.599562,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485
2,2020-05-15 00:30:00,4UPUqMRk7TRMgml,26.682055,24.427290,0.000000,4136001.0,0.000000,0.000000,9425.000000,2.429011e+06,...,26.512740,24.515678,25.317970,23.709773,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485
3,2020-05-15 00:45:00,4UPUqMRk7TRMgml,26.500589,24.420678,0.000000,4136001.0,0.000000,0.000000,9425.000000,2.429011e+06,...,26.494339,25.033410,25.217193,23.981148,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485
4,2020-05-15 01:00:00,4UPUqMRk7TRMgml,26.596148,25.088210,0.000000,4136001.0,0.000000,0.000000,7540.000000,2.429011e+06,...,26.420410,25.385702,25.065062,24.332798,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53851,2020-06-09 10:45:00,xoJJ8DcxJEcupym,31.264162,52.493663,0.870108,4136001.0,1035.178571,1010.557143,1719.857143,2.092877e+08,...,33.002392,0.000000,58.085779,0.000000,0.941356,0.0,792.854186,773.610377,778.313343,759.381538
53852,2020-06-09 11:00:00,xoJJ8DcxJEcupym,31.781721,53.449991,0.795815,4136001.0,1032.213333,1007.746667,2096.666667,2.092879e+08,...,32.501645,0.000000,50.182329,0.000000,0.548044,0.0,789.888948,770.799901,775.348105,756.571062
53853,2020-06-09 11:15:00,xoJJ8DcxJEcupym,32.175657,54.891657,0.794728,4136001.0,1186.320000,1157.613333,2378.800000,2.092882e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,943.995615,920.666568,929.454772,906.437728
53854,2020-06-09 11:30:00,xoJJ8DcxJEcupym,33.002392,58.085779,0.941356,4136001.0,1202.080000,1172.853333,2673.600000,2.092885e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,959.755615,935.906568,945.214772,921.677728


# Validation by comparison

In [4]:
gen_data['AC_POWER'][20000:20004]

20000    458.150000
20001    440.433333
20002    509.840000
20003    304.820000
Name: AC_POWER, dtype: float64

In [5]:
gen_data['AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1'][20000:20004]

20000    440.433333
20001    509.840000
20002    304.820000
20003    543.026667
Name: AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1, dtype: float64

In [6]:
a=gen_data[gen_data['SOURCE_KEY']=='1BY6WEcLGh8j5v7']
b=pd.concat([a[a['DATE_TIME']=='2020-05-17 13:00:00'],a[a['DATE_TIME']=='2020-05-16 13:00:00']], ignore_index=True)
b['AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1']

Series([], Name: AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1, dtype: float64)

In [7]:
b['AC_POWER']

Series([], Name: AC_POWER, dtype: float64)

In [8]:
groups[group].keys()

Index(['DATE_TIME', 'SOURCE_KEY', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE',
       'IRRADIATION', 'PLANT_ID', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD',
       'TOTAL_YIELD', 'ARTIFICIAL', 'DC_POWER_DIF_AVG_PER_ALL',
       'AC_POWER_DIF_AVG_PER_ALL', 'DATE', 'TIME',
       'DC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'DAILY_YIELD_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'DAILY_YIELD_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'TOTAL_YIELD_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'TOTAL_YIELD_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
     

# Other fields

In [9]:
#Group by day labels
'''
* label "FAULTY_according_to_DAILY_YIELD" - yield stays the same, while other inverters still increases
* label "INCORRECT_daily_yield" - yield per day decreases
* columns per inverter day average differences (total are done)
'''

uurimiseks= gen_data.copy()
groups = dict(list(uurimiseks.groupby("DATE")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_AVG_DIF_PER_DAY','AC_AVG_DIF_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_AVG_DIF_PER_DAY'] =  groups[group]['DC_POWER'] -np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_AVG_DIF_PER_DAY'] =  groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    gr_nr+=1
    '''
    Tundub et toodaks vale infot
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_DAY_SHIFT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data2=pd.DataFrame(columns =groups[group].keys())
    
    gen_data2=pd.concat([gen_data2,groups[group]], ignore_index=True)
    
gen_data2

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY
0,2020-05-15 00:00:00,4UPUqMRk7TRMgml,27.004764,25.060789,0.000000,4136001.0,0.000000,0.000000,9425.000000,2.429011e+06,...,24.420678,23.655607,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485,-315.628613,-308.445898
1,2020-05-15 00:15:00,4UPUqMRk7TRMgml,26.880811,24.421869,0.000000,4136001.0,0.000000,0.000000,9425.000000,2.429011e+06,...,25.088210,23.599562,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485,-315.628613,-308.445898
2,2020-05-15 00:30:00,4UPUqMRk7TRMgml,26.682055,24.427290,0.000000,4136001.0,0.000000,0.000000,9425.000000,2.429011e+06,...,25.317970,23.709773,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485,-315.628613,-308.445898
3,2020-05-15 00:45:00,4UPUqMRk7TRMgml,26.500589,24.420678,0.000000,4136001.0,0.000000,0.000000,9425.000000,2.429011e+06,...,25.217193,23.981148,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485,-315.628613,-308.445898
4,2020-05-15 01:00:00,4UPUqMRk7TRMgml,26.596148,25.088210,0.000000,4136001.0,0.000000,0.000000,7540.000000,2.429011e+06,...,25.065062,24.332798,0.000000,0.0,-297.597974,-290.887343,-178.256815,-174.417485,-315.628613,-308.445898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53851,2020-06-09 10:45:00,xoJJ8DcxJEcupym,31.264162,52.493663,0.870108,4136001.0,1035.178571,1010.557143,1719.857143,2.092877e+08,...,58.085779,0.000000,0.941356,0.0,792.854186,773.610377,778.313343,759.381538,781.595972,762.601487
53852,2020-06-09 11:00:00,xoJJ8DcxJEcupym,31.781721,53.449991,0.795815,4136001.0,1032.213333,1007.746667,2096.666667,2.092879e+08,...,50.182329,0.000000,0.548044,0.0,789.888948,770.799901,775.348105,756.571062,778.630734,759.791011
53853,2020-06-09 11:15:00,xoJJ8DcxJEcupym,32.175657,54.891657,0.794728,4136001.0,1186.320000,1157.613333,2378.800000,2.092882e+08,...,0.000000,0.000000,0.000000,0.0,943.995615,920.666568,929.454772,906.437728,932.737400,909.657677
53854,2020-06-09 11:30:00,xoJJ8DcxJEcupym,33.002392,58.085779,0.941356,4136001.0,1202.080000,1172.853333,2673.600000,2.092885e+08,...,0.000000,0.000000,0.000000,0.0,959.755615,935.906568,945.214772,921.677728,948.497400,924.897677


In [10]:
groups[group].keys()

Index(['DATE_TIME', 'SOURCE_KEY', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE',
       'IRRADIATION', 'PLANT_ID', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD',
       'TOTAL_YIELD', 'ARTIFICIAL', 'DC_POWER_DIF_AVG_PER_ALL',
       'AC_POWER_DIF_AVG_PER_ALL', 'DATE', 'TIME',
       'DC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'DAILY_YIELD_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'DAILY_YIELD_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'TOTAL_YIELD_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'TOTAL_YIELD_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
     

In [ ]:
# add inverter difference from average per that moment
'''
* columns "MOMENT_AVERAGE" - average value per this moment of inverter production info
* columns "DIF_FROM_AVERAGE" - inverter avg values in comparison with other inverters per this moment
'''

uurimiseks= gen_data2.copy()
groups = dict(list(uurimiseks.groupby("DATE_TIME")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_DIF_AVG_PER_DAY','AC_DIF_AVG_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_DIF_AVG_PER_MOMENT'] =  groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_MOMENT'] =  groups[group]['AC_POWER'] -np.mean(groups[group]['AC_POWER']) 

    gr_nr+=1
    ''' 
    tundub valet tulemust andvat
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_MOMENT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data3=pd.DataFrame(columns =groups[group].keys())
    
    gen_data3=pd.concat([gen_data3,groups[group]], ignore_index=True)
    
gen_data3

In [ ]:
#other labels adding
'''
* label "INCORRECT_total_yield" - yield per period decreases
* label "INCORRECT_according_to_DC_vs_AC" - DC and AC ratio is off
* label "INCORRECT_according_to_DC_AC_DAILY_YIELD" - DC and AC increases, while DAILY_YIELD does not
'''

In [ ]:
gen_data3.to_csv(path_or_buf='Data/Train_Plant2_added_features.csv')

In [ ]:
katse=pd.read_csv("Data/Train_Plant1_added_features.csv")

In [ ]:
katse